In [1]:
import numpy as np
import cv2
import os

!pip install ultralytics

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [29]:
!mkdir -p data/train\(zooniverse\)
!mkdir -p data/train\(region_growth\)
!mkdir -p data/val\(spoca\)
!mkdir -p data/val\(region\)

!cp -r scss-net/data/193_train/193\(zooniverse\)/imgs data/train\(zooniverse\)/images 
!cp -r scss-net/data/193_train/193\(zooniverse\)/masks data/train\(zooniverse\)/masks

!cp -r scss-net/data/193_train/193\(region_growth\)/imgs data/train\(region_growth\)/images
!cp -r scss-net/data/193_train/193\(region_growth\)/masks data/train\(region_growth\)/masks

!cp -r scss-net/data/193_test/masks\(spoca\) data/val\(spoca\)/masks
!cp -r scss-net/data/193_test/masks\(region\) data/val\(region\)/masks

!cp -r scss-net/data/193_test/imgs data/val\(spoca\)/images
!cp -r scss-net/data/193_test/imgs data/val\(region\)/images

In [8]:
def create_contours(mask):
    
    width = mask.shape[1]
    height = mask.shape[0]
    # ret, thresh = cv2.threshold(mask, 127, 255, 0)
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    
    coords = []
    for i, object in enumerate(contours):
        coords.append([])
        for point in object:
            new_x = (int(point[0][0])) / (width)
            new_y = (int(point[0][1])) / (height)
            coords[i].append((new_x, new_y))
    return coords

In [19]:
def convert_to_yolo(folder_path):
    masks_path = os.path.join(folder_path, 'masks')
    os.mkdir(os.path.join(folder_path, 'labels'))
    labels_path = os.path.join(folder_path, 'labels')

    png_files = [file for file in os.listdir(masks_path) if file.endswith('.png')]

    for png_file in png_files:
        input_path = os.path.join(masks_path, png_file)

        image_cv2 = cv2.imread(input_path, cv2.IMREAD_GRAYSCALE)

        coords_of_objects = create_contours(image_cv2)
    
        txt_file = os.path.splitext(png_file)[0] + '.txt'
        output_path = os.path.join(labels_path, txt_file)

        with open(output_path, 'w') as label_file:
            for i, object_coords in enumerate(coords_of_objects):
                coordinates_str = ' '.join(map(str, np.reshape(object_coords, -1)))
                string_to_write = f"0 {coordinates_str}"
                if i < (len(coords_of_objects) - 1):
                    string_to_write += "\n"
                label_file.write(string_to_write)

In [10]:
!python3 -m pip install --upgrade Pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 34.8 MB/s eta 0:00:00a 0:00:01
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Attempting uninstall: Pillow
    Found existing installation: Pillow 8.0.1
    Uninstalling Pillow-8.0.1:
      Successfully uninstalled Pillow-8.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
matrixprofile 1.1.10 requires protobuf==3.11.2, but you have protobuf 4.25.0 which is incompatible.


In [11]:
!pip3 install opencv-python

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [38]:
import cv2
from PIL import Image

def display_label(label_name):
    image = cv2.imread("./data/train(region_growth)/images/"+label_name+".png")

    with open("./data/train(region_growth)/labels/" + label_name + ".txt", 'r') as f:
        label = f.read()

    label_elements = label.strip().split(' ')

    coords = [(int(float(label_elements[i])*256), int(float(label_elements[i+1])*256)) for i in range(1, len(label_elements), 2)]
   
    for coord in coords:
        cv2.circle(image, coord, radius=0, color=(255, 0, 0), thickness=-1)
   
    cv2.imwrite("./test.png", image)


In [42]:
import cv2
   
def convert_to_grayscale(folder_path):
    images_path = os.path.join(folder_path, 'images')
    grayscale_images_path = os.path.join(folder_path, 'images-gray')
    os.makedirs(grayscale_images_path)
    
    png_files = [file for file in os.listdir(images_path) if file.endswith('.png')]
    
    for img_path in png_files:
        image = cv2.imread(images_path + "/" + img_path)
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
        cv2.imwrite(grayscale_images_path + "/" + img_path, gray_image)

        
        

In [43]:
convert_to_grayscale("./data/train(region_growth)")
convert_to_grayscale("./data/val(region_growth)")
convert_to_grayscale("./data/test(region_growth)")

In [14]:
from pathlib import Path

def remove_invalid_labels_with_images(data_path):
    images_path = os.path.join(data_path, 'images')
    labels_path = os.path.join(data_path, 'labels')

    txt_files = [file for file in os.listdir(labels_path) if file.endswith('.txt')]
    
    for txt_file in txt_files:
        with open(labels_path +"/"+ txt_file, 'r') as f:
            label = f.read()
            png_file = os.path.splitext(txt_file)[0] + '.png'
            if(len(label.split()) < 5):
                print("removing... " + png_file)
                Path(images_path+"/" + png_file).unlink()
                print("removing... " + txt_file)
                Path(labels_path+"/" + txt_file).unlink()
        
            

In [30]:
convert_to_yolo("./data/train(region_growth)")
convert_to_yolo("./data/test(region)")

In [31]:
# generate test set
import math
import os
import random
import shutil

def move_files(paths_list, dst_directory):
    for path in paths_list:
        src_path = path
        dst_path = os.path.join(dst_directory, os.path.basename(path))
        shutil.copyfile(src_path, dst_path)
        os.remove(src_path)

# we have 7873 images in region train and 354 in region val
test_num = math.floor(8227*0.1) # number of images in val set

train_path = "data/train(region_growth)"
test_path = "data/val(region_growth)"
images_path = os.path.join(train_path, "images")
test_images_path = os.path.join(test_path, "images")
test_labels_path = os.path.join(test_path, "labels")
test_masks_path = os.path.join(test_path, "masks")
os.makedirs(test_images_path, exist_ok=True)
os.makedirs(test_labels_path, exist_ok=True)
os.makedirs(test_masks_path, exist_ok=True)

image_paths = []
for root, _, files in os.walk(images_path):
    for file in files:
        if file.endswith(".png"):
            image_paths.append(os.path.join(root, file))

random.shuffle(image_paths)

test_images = image_paths[:test_num]
test_labels = [path.replace("images","labels").replace("png", "txt") for path in test_images]
test_masks = [path.replace("images","masks") for path in test_images]

move_files(test_images, test_images_path)
move_files(test_labels, test_labels_path)
move_files(test_masks, test_masks_path)

In [39]:
display_label("2012_01_31__22_59_43_84__SDO_AIA_AIA_193")

In [ ]:
# prepare SOHO test set
# install mega-cmd https://mega.io/cmd#downloadapps
import zipfile
!mega-get https://mega.nz/folder/M9klWZxK#QbxQMxVMCMtXS6nNzCtE7A/file/o4Nk0D7I
!mega-get https://mega.nz/folder/M9klWZxK#QbxQMxVMCMtXS6nNzCtE7A/file/5lEAwRjA
    
with zipfile.ZipFile("CH_test_imgs.zip","r") as zip_ref:
    zip_ref.extractall("testSOHOimgs")

with zipfile.ZipFile("CH_test_masks.zip","r") as zip_ref:
    zip_ref.extractall("testSOHOmasks")

In [16]:
!mkdir -p data/test\(SOHO\)
!mv testSOHOimgs data/test\(SOHO\)
!mv testSOHOmasks data/test\(SOHO\)

In [17]:
!mv data/test\(SOHO\)/testSOHOimgs data/test\(SOHO\)/images
!mv data/test\(SOHO\)/testSOHOmasks data/test\(SOHO\)/masks